< img src="logo.png" alt="University of Illinois" style="width: 50px;"/>

### Title ###
by: Chuanqi Chen  
by: Siyuan Teng 
* chuanqi2@illinois.edu
* siyuant2@illinois.edu


Project director: Richard Sowers
* <r-sowers@illinois.edu>
* <https://publish.illinois.edu/r-sowers/>

Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license

### Explanation###
4D Array of Occupancy by Street_Block, Date, Time and Price.  

In [ ]:
import glob
import numpy as np
import pandas as pd
import pickle
from datetime import datetime
import time

In [ ]:

df = pd.read_pickle("D:/myPython/data/parking_overall.p")

# Pre Test:    Total Observations = 3515680
len(df["STREET_BLOCK"].unique()) # 301
len(df["DATE"].unique())         # 730
# Time                           # 16


In [ ]:

# Hourly_Price
df["Total_Hour"] = (df['SESSION_END_DT'] - df['SESSION_START_DT']) / np.timedelta64(1, 'h')
Start_Time_In_Hour = df.SESSION_START_DT.dt.round('h').astype(str)
df["Start_Time_Only_Hour"] = Start_Time_In_Hour.str.split(" ", n=1, expand=True)[1] # Time Axis by Dataset

# Filter from 6am to 9pm
df = df[ ("05:30:00"<=df["Start_Time_Only_Hour"]) & (df["Start_Time_Only_Hour"]<="21:30:00") ]

by_SbDtTime = df.groupby(["STREET_BLOCK", "DATE", "Start_Time_Only_Hour"]).sum()
by_SbDtTime['Hourly_Rate'] = by_SbDtTime['NET_PAID_AMT'] / by_SbDtTime['Total_Hour']

# Use the Mechenism and Details of pivot() and stack()
Price_stacked = by_SbDtTime.reset_index(["STREET_BLOCK", "DATE", "Start_Time_Only_Hour"]).\
    pivot_table(index=["STREET_BLOCK", "DATE"], columns="Start_Time_Only_Hour", values="Hourly_Rate").\
    stack(dropna=False)   # !!!! Right


In [ ]:
# Occupancy
def number_of_cars_621(df):
    time_in_out = pd.concat([df["SESSION_START_DT"], df["SESSION_END_DT"]], ignore_index=True).sort_values()
    ones = pd.Series(np.where((time_in_out.index).isin(range(df.shape[0])), 1, -1), index=time_in_out)
    ones_HourIdx = pd.Series( ones.index).astype(str).str.split(" ", n=1, expand=True )[1]

    # Filter from 6am to 9pm
    new_ones = ones[ ("05:30:00" <= ones_HourIdx).values & (ones_HourIdx <= "21:30:00").values ]

    if len(new_ones) == 0:  # In case of Empty Dataframe After Filter
        return pd.Series(0, index=range(6,22))  # Must return Series to Match the form


    new_ones_HourIdx = pd.Series(new_ones.index).astype(str).str.split(" ", n=1, expand=True)[1]

    occupancy_By_TimeTick = []   # Store Occupancy according to Time Tick
    for Time_Tick in range(6,22):  # Time Axis by User
        if Time_Tick <= 9:
            Tick = "0" + str(Time_Tick) + ":30:00"
        else:
            Tick = str(Time_Tick) + ":30:00"
        occupancy_By_TimeTick.append( new_ones[ (new_ones_HourIdx <= Tick).values ].sum() ) # Sum "ones" until Tick

    return pd.Series(occupancy_By_TimeTick, index=range(6,22))

occupancy_By_SbDtTime = df.groupby(["STREET_BLOCK", "DATE"]).apply(number_of_cars_621)

# occupancy_By_SbDtTime = pd.read_csv("D:/myPython/data/occupancy_By_SbDtTime.csv").set_index(["STREET_BLOCK", "DATE"])
occupancy_stacked = occupancy_By_SbDtTime.stack(dropna=False)



In [ ]:
# Final DataFrame
Final_df = occupancy_stacked.reset_index(level=[0,1,2])
Final_df.columns = ["Street_Bolck", "Date", "Time", "Occupancy"]
Final_df["Price"] = Price_stacked.values  # saved


In [ ]:
# Make the Date Same
Final_df_stacked =   Final_df.pivot_table(index="Street_Bolck", columns=["Date", "Time"], values=["Occupancy", "Price"]).\
    stack(["Date","Time"],dropna=False)

tensor_frame = Final_df_stacked.reset_index(["Street_Bolck", "Date", "Time"])  # Saved  3515680 obs


In [ ]:
# Make the Price Same
tensor_frame["Price"][ (tensor_frame["Price"]<=0) | (tensor_frame["Price"]>=50) ] = np.nan
tensor_frame["Price"] = tensor_frame["Price"].round()

len(tensor_frame["Price"].unique())

# Choose "01ST ST 200", "2011-04-01", 6, NaN
# to create a complete "Real Dataframe Structure"
df_cell = pd.DataFrame({"Street_Bolck":["01ST ST 200"]*51,
                        "Date":["2011-04-01"]*51,
                        "Time":[6]*51,
                        "Occupancy":[np.nan]*51,
                        "Price":sorted(tensor_frame["Price"].unique())})
df_cell.iloc[0,3] = 0

tensor_frame2_temp = pd.concat([df_cell, tensor_frame.drop(0)], ignore_index=True)

tensor_frame2_temp["Price"][ tensor_frame2_temp["Price"].isna() ] = -1


# This is Complete Final Data in the form of Series
series_final = tensor_frame2_temp.pivot_table(index=["Street_Bolck", "Date", "Time"], columns="Price", values="Occupancy", dropna=False).stack(dropna=False)


In [ ]:
# Make 4D Array from "series_final"
series_final.values.reshape(51, 16, 730, 301)
